In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/petebrown/data-updater/main/data/results.csv", parse_dates=["game_date"])

In [3]:
df["pregame_date"] = df["game_date"] - pd.Timedelta(days=1)

In [4]:
def construct_url(competition, game_date):
    game_date = pd.to_datetime(game_date)
    day = game_date.day
    month = game_date.month_name().lower()
    year = game_date.year
    division = competition.lower().replace(" ", "-").replace("(", "").replace(")", "")

    if day < 10:
        day = f'0{day}'
    url = f"https://www.11v11.com/league-tables/{division}/{day}-{month}-{year}/"
    
    return url

In [5]:
df['url'] = df.apply(lambda x: construct_url(x.competition, x.pregame_date), axis = 1)

In [26]:
lge_dates = df.query("game_type == 'League' & ssn_comp_game_no > 1 & league_tier <= 4")[["pregame_date", "game_date", "ssn_comp_game_no", "url"]].rename(columns={"ssn_comp_game_no": "game_no"})

In [14]:
lge_dates = lge_dates.query("game_date > '2018-08-01'")

In [16]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}

urls = lge_dates['url'].to_list()

tables = []
for url in urls:
    r = requests.get(url, headers = headers)
    doc = BeautifulSoup(r.text, 'html.parser')

    table = pd.read_html(StringIO(str(doc)))[0]

    table["pos"] = table.index + 1
    table["url"] = url
    
    table = table[['pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'Pts', 'url']]
    
    tables.append(table)

tables_df = pd.concat(tables)

In [27]:
tables_df.merge(lge_dates, how="left", on="url").to_csv("./league_tables.csv", index=False)